In [ ]:
# Map the dropdown value to enable_TVDL_strategy
if TT.value == '3':
    enable_TVDL_strategy = 1
elif TT.value == '1':
    enable_TVDL_strategy = 0
elif TT.value == '5':
    enable_TVDL_strategy = 2
def check_strength(training_data, desired_strength):
    # Check if any strength in the selected data is >= desired_strength
    return any(s["Shear Modulus"] >= desired_strength for s in training_data)

def select_and_format_data(data, desired_strength):
    while True:
        # Randomly select 4 data points
        selected_data = data.sample(n=4)  # Removed random_state for true randomness

        # Parse selected data into the required format
        training_data_formatted = []
        tested_formulations = []
        for _, row in selected_data.iterrows():
            # Ensure these column names match your DataFrame exactly
            M_atom = row["M-site element"]
            A_atom = row["A-site element"]
            X_atom = row["X-site element"]
            # Extract Fly Ash/GGBFS ratio and curing method from materials as done previously
            #fa_ggbfs = materials.split(",")[0].split("-")[1]
            #curing_method = materials.split(",")[-1].strip()
            #curing_method = curing_method.replace(" (Rao et al. 2018)", "").replace(" (Rao et al.)", "")
            #if curing_method == 'Ambient curing':
                #curing_method = 0
            #elif curing_method == 'Heat curing':
                #curing_method = 1
            #else:
                #print('Unknown curing method error in select_and_format_data')
            strength = row["G (Shear modulus)"]
            
            # Certainly needs to be a change here (me)

            training_str = f"M = {M_atom}, A = {A_atom}, X = {X_atom} resulted in a shear modulus of {strength} MPa"
            training_data_formatted.append({'formatted_str': training_str, 'Shear Modulus': strength})
            tested_str = f"M = {M_atom}, A = {A_atom}, X = {X_atom}"
            tested_formulations.append(tested_str) 

        # Check if any selected data point's strength >= desired_strength
        if not check_strength(training_data_formatted, desired_strength):
            # Return both the formatted training data and the tested formulations
            return [d['formatted_str'] for d in training_data_formatted], tested_formulations
        
def find_matching_result(df, suggestion):
    if suggestion:
        # Create the suggestion string in the same format as the formulation strings
        suggestion_str = f'The combination is M = {suggestion["M"]}, A = {suggestion["A"]}, X = {suggestion["X"]}'
        TrainingDat = f'M = {suggestion["M"]}, A = {suggestion["A"]}, X = {suggestion["X"]}'
        
        # Look for a match in the DataFrame
        match = df[df["Combination"].str.lower() == suggestion_str.lower()]
        
        # If a match was found, return the lab result and TrainingDat
        if not match.empty:
            return match.iloc[0]["Shear Modulus"], TrainingDat
        
        # If no match was found, print the suggestion string for debugging
        else:
            print("No match found for suggestion string: ", suggestion_str)

    # If no suggestion provided or no match found, return None
    return None, None


import re
import json

def parse_solution(response):
    # Initialize a dictionary to hold the solution
    solution = {}

    # Function to normalize key names
    def normalize_key(key):
        # Normalize common variations to a standard form
        key_map = {
            'M-site element': 'M',
            'A-site element': 'A',
            'X-site element': 'X',
        }
        for known_key, normalized_key in key_map.items():
            if known_key in key.lower().replace(" ", ""):
                return normalized_key
        return None

    # Try to parse the response as JSON
    #try:
        #json_data = json.loads(response)
        #for key, value in json_data.items():
            #normalized_key = normalize_key(key)
            #if normalized_key:
                #solution[normalized_key] = str(value)
        #if solution:  # If we successfully extracted data
            #return solution
    #except json.JSONDecodeError:
        # If JSON parsing fails, proceed with regex parsing for the plain text format
    keys = ['M', 'A', 'X']  # Updated to include 'curing'
    for key in keys:
        #if key != 'D':  # For 'curing', we might need a different approach
            match = re.search(fr"{key} = (.*?)(,|$)", response, re.IGNORECASE)
            if match:
                value = match.group(1).strip()
                solution[key] = value
            else:
                # If any key wasn't found using regex, return None
                return None
        #else:
            # Handle 'curing' specifically based on the presence of keywords
            #if "ambient" in response.lower():
                #solution["D"] = "Ambient curing"
            #elif "heat" in response.lower():
                #solution["D"] = "Heat curing"
            #else:
                # If 'curing' condition is not met, return None
                #return None
    return solution  # Return the solution dictionary if all keys were found with regex

    # Return None if neither JSON nor regex parsing succeeded
    #return None

    
def format_response_to_model(lab_result):
    """
    Given a lab result, format a response message to the model.
    """
    return f"We've achieved a performance of {lab_result['G (Shear modulus)']}. Let's try to do better!"


df = pd.read_csv('Data/alloys.csv')

# Initialize empty DataFrame
formulation_df = pd.DataFrame(columns=["Combination", "Shear Modulus"])

# Loop through each row in the original data
for idx, row in df.iterrows():
    
    # Get necessary attributes from row
    M = row["M-site element"]
    A = row["A-site element"]
    X = row["X-site element"]

    # Compressive strength
    strength = row["G (Shear modulus)"]
    
    # Create formulation string in the same format as the model's output
    formulation = f'The combination is M = {M}, A = {A}, X = {X}'
    
    # Append the formulation and its respective strength to the new DataFrame
    new_row = pd.DataFrame({"Combination": [formulation], "Shear Modulus": [strength]})
    formulation_df = pd.concat([formulation_df, new_row], ignore_index=True)

def handle_openai_error(exception):
    if isinstance(exception, openai.error.RateLimitError):
        print(f"Rate limit error. Will retry after {exception.wait_seconds} seconds.")
        time.sleep(exception.wait_seconds)
    elif isinstance(exception, openai.error.InvalidRequestError):
        print(f"Invalid request: {str(exception)}")
    elif isinstance(exception, openai.error.AuthenticationError):
        print(f"Authentication error: {str(exception)}")
    elif isinstance(exception, openai.error.ServiceUnavailableError):
        print(f"Service unavailable error. Retrying after a delay...")
        time.sleep(5)  # Sleep for 5 seconds before retrying
    elif isinstance(exception, openai.error.APIError):
        print(f"API error: {str(exception)}. Retrying after a delay...")
        time.sleep(5)  # Sleep for 5 seconds before retrying
    elif isinstance(exception, openai.error.Timeout):
        print(f"Timeout error: {str(exception)}. Retrying after a longer delay...")
        time.sleep(10)  # Sleep for 10 seconds before retrying
    else:
        raise exception
        
# -> here we also set the API parameters, such as temperature, etc.

def call_openai_api(messages,temp, max_retries=5, delay=5):
    for i in range(max_retries):
        try:
            response = openai.chat.completions.create(
                model= model_dropdown.value,
                temperature=temp,
                messages=messages,
                max_tokens=500,
                n=1
            )
            return response
        except openai.error.OpenAIError as e:
            
            handle_openai_error(e)
            if i < max_retries - 1:  # i is zero indexed
                time.sleep(delay)  # wait before trying again
                continue
            else:
                raise
                
# Load the text from the file
if  prompt_dropdown.value == 'ternary':
    with open('Prompts/prompts_ID_ternary.txt', 'r') as f:
        lines = f.read().splitlines()

if  prompt_dropdown.value == 'terncontext':
    with open('Prompts/prompts_ID_ternarycontext.txt', 'r') as f:
        lines = f.read().splitlines()
# Store the contents in separate variables
#instructions_text = lines[0]
DA_role_text = lines[0]
context_text = lines[1]
iterate_text = lines[2]
VM_role_text = lines[3]


  
# Create the widgets with the loaded text
layout = widgets.Layout(width='auto', height='200px')  # adjust the height and width as needed
#instructions_prompt = widgets.Textarea(value=instructions_text, description='Instructions:', layout=layout)
DA_role_prompt = widgets.Textarea(value=DA_role_text, description='DA Instuctions:', layout=layout)
VM_role_prompt = widgets.Textarea(value=VM_role_text, description='VM Instuctions:', layout=layout)
context_prompt = widgets.Textarea(value=context_text, description='Design Knowledge:', layout=layout)

iterate_prompt = widgets.Textarea(value=iterate_text, description='User:', layout=layout)


# Run to display the text box widgets and the save button

# Set the 'flex' property for each widget inside main_layout
iterate_prompt.layout.flex = '2'       # Adjust the value as needed

#Instructions_box = widgets.HBox([instructions_prompt],layout=widgets.Layout(width='100%', height='100px'))
Iterate_box = widgets.HBox([iterate_prompt],layout=widgets.Layout(width='100%', height='100px'))

title = widgets.HTML("<h2>Main Prompts</h2>")
Prompts = widgets.VBox([title ,DA_role_prompt, VM_role_prompt, Iterate_box, context_prompt])

temperatures_str = model_temperatures.value
# Split the string using commas as a delimiter and convert to float numbers
temperatures = [float(t) for t in temperatures_str.split(',')]
budget = num_development.value
NrOfExper = num_experiments.value

desired_strength = formulation_df["Shear Modulus"].quantile(targ_quant.value/100)

num_entries_above_desired = (formulation_df["Shear Modulus"] >= desired_strength).sum()

# Print the result

print('SUMMARY\n' +'The design target is to achieve a shear modulus of ',desired_strength, 'MPa within ',num_development.value,' development cycles.\n' +
      'The Experiment is repeated ',num_experiments.value,' times using the ',model_dropdown.value, ' model and the prompt strategy: ',prompt_dropdown.value,'.')
print("There are ", num_entries_above_desired,' combinations above or equal to desired_strength.')


def extract_formulations_from_training_data(training_data):
    pattern = re.compile(r'M\s*=\s*(\w+),\s*A\s*=\s*(\w+),\s*X\s*=\s*(\w+)', re.IGNORECASE)
    training_formulations = [match.group(0) for data in training_data for match in [pattern.search(data)] if match]
    return training_formulations

NameError: name 'TT' is not defined